## Import 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegressionCV

from imblearn.under_sampling import RandomUnderSampler
import numpy as np


## Reading Input

In [ ]:
df = pd.read_csv('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
df.head(5)

## Data Exploration

### Check for linerity

First we will check if features correlate with each other. 
Easy can use function pairplot to visualize it.

In [ ]:
sns.pairplot(df[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']])

As we can see some features have linear dependency. In this case we need to check there correlation,
in case if it is high, than we will drop on of it or using two of them create new feature.

Check linear relationship between the features and the target.

In [ ]:
sns.pairplot(df, x_vars=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'], y_vars='quality')

### Correlation Matrix

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

correlation = df.corr()
sns.heatmap(correlation, cmap='coolwarm', annot=True, fmt=".2f")

plt.xticks

### Check dataset classes balance

In [ ]:
sns.countplot(df['quality'], data=df)

As we can see, labels are not balanced. So we need first to transform it to binary labels and unsample it

In [ ]:
labels = ['bad', 'good']
bins = [2,6.5,8]
df['quality'] = pd.cut(df['quality'], bins=bins, labels=labels)

In [ ]:
label_enc = LabelEncoder()
df['quality'] = label_enc.fit_transform(df['quality'])

#### Upsample dataset

In [ ]:
X = df.drop('quality', axis=1)
y = df['quality']

In [ ]:
resampler = RandomUnderSampler(random_state=0, replacement=True)
resampler.fit(X,y)
X_resampled, y_resampled = resampler.fit_resample(X,y)

In [ ]:
sns.countplot(y_resampled)

Now we have balanced dataset

## Data PreProcessing

In [ ]:
X_resampled.isnull().sum()

Split dataset on train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

Normalize data

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

## Model training

### Logistic Regression

In [ ]:
model2 = LogisticRegressionCV(cv=5,random_state=0)
model2.fit(X_train,y_train)
pred2 = model2.predict(X_test)
print(mean_absolute_error(y_test,pred2))
print(accuracy_score(y_test,pred2))

In [ ]:
print(classification_report(y_test,pred2))

### Random Forest Classifier

In [ ]:
model1 = RandomForestClassifier(n_estimators=50, max_depth=10, max_features='log2')

param_grid = {
    'n_estimators' : [50,1000],
    'max_depth': [1,100],
    'max_features': ['auto', 'sqrt', 'log2']
}


CV_rfc = GridSearchCV(estimator=model1, param_grid=param_grid, cv= 5)
CV_rfc.fit(X, y)
print (CV_rfc.best_params_)

In [ ]:
model1 = RandomForestClassifier(n_estimators=50, max_depth=100, max_features='log2')

model1.fit(X_train,y_train)
pred1 = model1.predict(X_test)
print(mean_absolute_error(y_test,pred1))
print(accuracy_score(y_test,pred1))

In [ ]:
print(classification_report(y_test,pred1))

### Gradient Boosting

In [ ]:
clf = GradientBoostingClassifier(n_estimators=40,  learning_rate=0.2, max_depth=1)

clf.fit(X_train,y_train)
pred3 = clf.predict(X_test)
print(mean_absolute_error(y_test,pred3))
print(accuracy_score(y_test,pred3))

In [ ]:
print(classification_report(y_test,pred3))